In [2]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from collections import deque

bot_points = 0
user_points = 0

# Mapping
moves = ['rock', 'paper', 'scissors']
move_to_num = {'rock': 0, 'paper': 1, 'scissors': 2}
num_to_move = {v: k for k, v in move_to_num.items()}

# Store user history
user_moves = []

# AI Model
def build_model():
    model = Sequential([
        Embedding(input_dim=3, output_dim=4, input_length=1),
        Flatten(),
        Dense(10, activation='relu'),
        Dense(3, activation='softmax')
    ])
    model.compile(optimizer=Adam(0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_model()
trained = False

# Predict and counter
def predict_and_counter(last_user_move):
    if not trained or last_user_move is None:
        return random.randint(0, 2)  # Random if no model or no last move
    input_arr = np.array([[last_user_move]])
    prediction = model.predict(input_arr, verbose=0)
    predicted_move = np.argmax(prediction)
    return (predicted_move + 1) % 3  # AI counters user's next move

# Game logic
def winner(ai, user):
    if ai == user:
        return "Draw"
    elif (ai - user) % 3 == 1:
        return "AI Wins"
        bot_points = bot_points+1
    else:
        return "You Win"
        user_points = user_points+1

# Game loop
rounds = 0
last_move = None

print("🎮 Rock-Paper-Scissors (AI-Powered) — Type 'exit' to quit.\n")

while True:
    user_input = input("Your move (rock/paper/scissors): ").lower()
    if user_input == 'exit':
        print("Here are the scores! : User points = "+user_points+" _____ Bot points = "+bot_points)
        break
    if user_input not in move_to_num:
        print("Invalid move. Try again.")
        continue

    user_move = move_to_num[user_input]
    ai_move = predict_and_counter(last_move)

    print(f"AI chose: {num_to_move[ai_move]}")
    print(winner(ai_move, user_move))

    user_moves.append(user_move)
    last_move = user_move
    rounds += 1

    if rounds % 10 == 0 and len(user_moves) > 1:
        print("\nretraining AI...\n")
        X, y = [], []
        for i in range(len(user_moves) - 1):
            X.append(user_moves[i])
            y.append(user_moves[i + 1])
        X = np.array(X)
        y = np.array(y)
        model.fit(X, y, epochs=15, verbose=0)
        trained = True


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


🎮 Rock-Paper-Scissors (AI-Powered) — Type 'exit' to quit.

Your move (rock/paper/scissors): rock
AI chose: rock
Draw
Your move (rock/paper/scissors): paper
AI chose: rock
You Win
Your move (rock/paper/scissors): paper
AI chose: rock
You Win
Your move (rock/paper/scissors): rock
AI chose: paper
AI Wins
Your move (rock/paper/scissors): scissors
AI chose: rock
AI Wins
Your move (rock/paper/scissors): paper
AI chose: scissors
AI Wins
Your move (rock/paper/scissors): rock
AI chose: paper
AI Wins
Your move (rock/paper/scissors): paper
AI chose: paper
Draw
Your move (rock/paper/scissors): rock
AI chose: paper
AI Wins
Your move (rock/paper/scissors): paper
AI chose: scissors
AI Wins

🔁 Retraining AI...

Your move (rock/paper/scissors): paper
AI chose: paper
Draw
Your move (rock/paper/scissors): scissors
AI chose: paper
You Win
Your move (rock/paper/scissors): rock
AI chose: scissors
You Win
Your move (rock/paper/scissors): scissors
AI chose: scissors
Draw
Your move (rock/paper/scissors): rock
